# How we caught the Circle Line rogue train with data

_Note: This is meant to be a companion to our post on the Data.gov.sg blog (https://blog.data.gov.sg/how-we-caught-the-circle-line-rogue-train-with-data-79405c86ab6a). The code here was written on November 5, 2016 - the actual day  when we were working on SMRT data to identify the cause of the Circle Line incidents. We acknowledge that there could be inefficiencies and welcome any comments and further sharing._

Authors: Lee Shangqian, Daniel Sim and Clarence Ng, Data Science Division, GovTech Singapore

We were given a data dump containing the following information:
- Date and time of of each incident
- Location of incident
- ID of train involved
- Direction of train

We started by cleaning the data.

As usual, the first step is to import some useful python libraries.

In [1]:
import math
import xlrd
import itertools as it
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Make data extracts smaller for online consumption
pd.set_option("display.max_rows", 5)

# Suppress warnings for online consumption
pd.options.mode.chained_assignment = None

We then extract the useful parts from the raw data.

In [3]:
dfincidents_0 = pd.read_excel('CCL EVAC E-brake occurrences hourly update_mod.xlsx', sheetname='Aug Sep')
dfincidents_1 = pd.read_excel('CCL EVAC E-brake occurrences hourly update_mod.xlsx', sheetname='Nov')

# Incident data for Nov had different format
dfincidents_1['Time'] = dfincidents_1['Time'].str.strip('hrs').str.strip(' ')
dfincidents_1['Time'] = pd.to_datetime(dfincidents_1['Time'], format='%H%M').dt.time

dfincidents = pd.concat([dfincidents_0, dfincidents_1])

# Reset the index because they were concatenated from two data sources
dfincidents.reset_index(inplace=True, drop=True)

In [4]:
def datetime_from_date_and_time(row):
    """
    Combines the date column and time column into a single column
    """
    d = row['Date']
    t = row['Time']
    
    return datetime(
        d.year, d.month, d.day,
        t.hour, t.minute, t.second
    )

# Add DateTime to the data for easier visualization
dfincidents['DateTime'] = dfincidents.apply(datetime_from_date_and_time, axis=1)
dfincidents

,S/N,S/N.1,Date,Traffic Date,PV,Time,Bound,Station from,Station to,Event,Remarks,DateTime
0,1,1,2016-08-28,2016-08-28,PV40,19:32:00,OT,KRG,ONH,EB,point track,2016-08-28 19:32:00
1,2,2,2016-08-28,2016-08-28,PV53,19:39:00,OT,LBD,PPJ,EB,point track,2016-08-28 19:39:00
...,...,...,...,...,...,...,...,...,...,...,...,...
257,57,42,2016-11-04,2016-11-04,PV13,22:29:00,IT,SDM,MBT,EB,NaN,2016-11-04 22:29:00
258,58,43,2016-11-05,2016-11-05,PV43,00:07:00,IT,TLB,HBF,EVAC,"Withdrawal train, no pax on-board",2016-11-05 00:07:00


We could not find any obvious answers in our initial exploratory analysis, so our next step was to incorporate multiple dimensions. Our intention was to plot the incidents on a Marey Chart.

First, we converted the station names from their three-letter codes to a number:
- Marina Bay to before Promenade: 0 to 1.5
- Dhoby Ghaut to HarbourFront: 2 to 29

If the incident occurred between two stations, it will be denoted as 0.5 + the lower of the two station numbers. For example, If an incident happened between HarbourFront (number 29) and Telok Blangah (number 28), the location will be “28.5”. This makes it easy for us to plot the points along the horizontal axis.

In [5]:
stations=("MRB,BFT,DBG,BBS,EPN,PMN,NCH,SDM,MBT,DKT,PYL,MPS,TSG,BLY,SER,"
          "LRC,BSH,MRM,CDT,BTN,FRR,HLV,BNV,ONH,KRG,HPV,PPJ,LBD,TLB,HBF").split(',')


def loc_id(station1, station2 = None):
    """
    Translates a 3-letter station code to a number,
    or a pair of 3-letter station codes to a number.
    Single stations are represented as whole numbers.
    Locations between stations are represented with a .5.
    
    Example:
    loc_id('MRB')         # 0 (Marina Bay)
    loc_id('MRB', 'BFT')  # 0.5 (Between Marina Bay and Bayfront)
    loc_id('DBG')         # 2 (Dhoby Ghaut)
    loc_id('HBF')         # 29
    loc_id('HBF', 'TLB')  # 28.5 (Between Harbourfront and Telok Blangah)
    loc_id('HBF', 'DBG')  # throws and error, because these stations are not adjacent
    """
    if station2 == None or station2 == 'nan' or (type(station2) is float and math.isnan(station2)):
        # Single stations
        return stations.index(station1)
    
    else: # Pairs of stations -- take the average to get the 0.5
        stn1_index = stations.index(station1)
        stn2_index = stations.index(station2)
        
        # Handle the branch at Promenade
        if (set(['PMN', 'EPN']) == set([station1, station2])):
            return float(stations.index('EPN')) + 0.5
        elif set(['PMN', 'BFT']) == set([station1, station2]):
            return float(stations.index('BFT')) + 0.5
        else:
            # Require station pairs to be adjacent stations
            assert(math.fabs(stn1_index - stn2_index) == 1)
            return float(stn1_index + stn2_index) / 2

And then we computed the numeric location IDs…

In [7]:
def loc_id_from_stations(row):
    try:
        # This handles entries with both "Station from" and "Station to"
        # and entries with only "Station from"
        return loc_id(row['Station from'], str(row['Station to']))
    except ValueError:
        # Some entries only have "Station to" but no
        # "Station from"
        return loc_id(row['Station to'])

And added that to the dataset:

In [8]:
# Select only some columns that we are interested in
sel_dfincidents = dfincidents[['DateTime', 'PV', 'Bound', 'Station from', 'Station to', 'Event', 'Remarks']]

# Add the location ID into the dataset
sel_dfincidents['LocID'] = sel_dfincidents.apply(loc_id_from_stations, axis=1)
sel_dfincidents

,DateTime,PV,Bound,Station from,Station to,Event,Remarks,LocID
0,2016-08-28 19:32:00,PV40,OT,KRG,ONH,EB,point track,23.5
1,2016-08-28 19:39:00,PV53,OT,LBD,PPJ,EB,point track,26.5
...,...,...,...,...,...,...,...,...
257,2016-11-04 22:29:00,PV13,IT,SDM,MBT,EB,NaN,7.5
258,2016-11-05 00:07:00,PV43,IT,TLB,HBF,EVAC,"Withdrawal train, no pax on-board",28.5


With the data processed, we were able to create a scatterplot of all the emergency braking incidents. 

We noticed that the sequence of breakdowns seem to move “backwards”. In other words, when a train gets hit by the interference, another train behind moving in the same direction gets hit soon after.

## What moves backwards in a tunnel? 

At this point, it still wasn’t clear that a single train was the culprit. 

But what’s the most obvious thing that moves down a train line along a tunnel? Could it be a train moving in the opposite direction? We decided to test this “rogue train” hypothesis.

We knew that the travel time between stations along the Circle Line ranges between two and four minutes. This means we could group all emergency braking incidents together if they occur up to four minutes apart.

In [9]:
def same_cascade(i, j):
    """
    Given a pair of incidents (i,j), returns true if:
    
        t <= d * 4 mins
    
    where t is  the time difference between occurrences
    and d is the distance (measured by difference in location ID).
    
    Moreover, we consider the track direction, and only consider
    incidents that are "moving backwards".
    """
    
    # If trains are not travelling in the same direction
    # they cannot be due to the same "backward moving" interference
    # source.
    # (Note: This was the hypothesis when this code was written.
    # It turned out that the rogue train could affect all
    # trains in the vicinity, not just in the opposite track)
    if i["Bound"] != j["Bound"] or \
        i["Bound"] not in ['IT', 'OT']:
        return False
    
    # time difference in minutes
    time_difference = (i["DateTime"] - j["DateTime"]) / np.timedelta64(1, 'm')
    location_difference = i["LocID"] - j["LocID"]
    
    if location_difference == 0:
        return False
    
    ratio = time_difference / location_difference
    
    if i["Bound"] == 'OT':
        return ratio > 0 and ratio < 4
    elif i["Bound"] == 'IT':
        return ratio < 0 and ratio > -4

We found all incident pairs that satisfied this condition:

In [10]:
incidents = sel_dfincidents.to_records()
# (a, b, c, d, ...) --> ((a,b), (a,c), (a,d), ..., (b,c), (b,d), ..., (c,d), ...)
incident_pairs = list(it.combinations(incidents, 2))

related_pairs = [ip for ip in incident_pairs if same_cascade(*ip)]
related_pairs = [(i[0], j[0]) for i,j in related_pairs]

We then grouped all related pairs of incidents into larger sets using a disjoint-set data structure. This allows us to group incidents that could be linked to the same “rogue train”.

In [11]:
def pairs_to_clusters(pairs):
    """
    A quick-and-dirty disjoint-set data structure. 
    But this works fast enough for 200+ records.
    Could be better.
    
    Example input:
    (1,2), (2,3), (4,5)
    
    Output:
    1: {1,2,3}
    2: {1,2,3}
    3: {1,2,3}
    4: {4,5}
    5: {4,5}
    """
    the_clusters = dict()

    for i,j in pairs:
        if i not in the_clusters:
            if j in the_clusters:
                the_clusters[j].add(i)
                the_clusters[i] = the_clusters[j]
            else:
                the_clusters[i] = set(list([i, j]))
                the_clusters[j] = the_clusters[i]
        else:
            if j in the_clusters:
                if the_clusters[i] is not the_clusters[j]: # union the two sets
                    for k in the_clusters[j]:
                        the_clusters[i].add(k)
                        the_clusters[k] = the_clusters[i]
                else: # they are already in the same set
                    pass
            else:
                the_clusters[i].add(j)
                the_clusters[j] = the_clusters[i]
    
    return the_clusters

Then we applied our algorithm to the data:

In [12]:
clusters = pairs_to_clusters(related_pairs)
# Show each set only once
clusters = [v for k,v in clusters.items() if min(v) == k]
clusters[0:10]

[{0, 1},
 {2, 4},
 {5, 6, 7},
 {8, 9},
 {18, 19, 20},
 {21, 22, 24, 26, 27},
 {28, 29, 30, 31, 32, 33, 34},
 {42, 44, 45},
 {47, 48},
 {51, 52, 53, 56}]

Next, we calculated the percentage of the incidents that can explained by our clustering algorithm.

In [13]:
# count % of incidents occurring in a cluster
all_clustered_incidents = set()

for i,clust in enumerate(clusters):
    all_clustered_incidents |= clust

(len(all_clustered_incidents),
 len(incidents),
 float(len(all_clustered_incidents)) / len(incidents))

(189, 259, 0.7297297297297297)

What it means: Of the 259 emergency braking incidents in our dataset, 189 cases — or 73% of them — could be explained by the “rogue train” hypothesis.

We went on to process the historical location data of trains on the Circle Line and concluded that more than 95% of incidents from August to November could be explained by the rogue train hypothesis. 